In [19]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn import Sequential
from torchvision import datasets, transforms
import matplotlib.pyplot as plt



In [20]:
import pyvista as pv
from matplotlib.style.core import library


print(pv.__version__)

0.45.3


In [21]:
import numpy as np
import glob

from data_processor import sorted_vtk
"""files = sorted(glob.glob("VTK/cavity_*.vtk"), key = lambda x: int(x.split("_")[-1].split(".")[0]))
files_2 = sorted(glob.glob("VTK_Uin1_Re1000/cavity_*.vtk"), key = lambda x: int(x.split("_")[-1].split(".")[0]))
print(f"{len(files_2)} files!")"""

cases = [
    {"files": sorted_vtk("VTK_Uin0_1_Re100/cavity_*.vtk"), "global_attr": [0.1, 100.0]},
    {"files": sorted_vtk("VTK_Uin0_5_Re50/cavity_*.vtk"), "global_attr": [0.5, 50.0]},
    {"files": sorted_vtk("VTK_Uin1_5_Re150/cavity_*.vtk"), "global_attr": [1.5, 150.0]},
    {"files": sorted_vtk("VTK_Uin1_5_Re1500/cavity_*.vtk"), "global_attr": [1.5, 1500.0]},
    {"files": sorted_vtk("VTK_Uin1_25_Re1250/cavity_*.vtk"), "global_attr": [1.25, 1250.0]},
    {"files": sorted_vtk("VTK_Uin1_Re100/cavity_*.vtk"), "global_attr": [1.0, 100.0]},
    {"files": sorted_vtk("VTK_Uin1_Re1000/cavity_*.vtk"), "global_attr": [1.0, 1000.0]},
    {"files": sorted_vtk("VTK_Uin2_0_Re200/cavity_*.vtk"), "global_attr": [2.0, 200.0]},
    {"files": sorted_vtk("VTK_Uin0_05_Re10/cavity_*.vtk"), "global_attr": [0.05, 10.0]},
    {"files": sorted_vtk("VTK_Uin4_Re100/cavity_*.vtk"), "global_attr": [4.0, 100.0]}
]


In [22]:
import importlib
import get_node_info
importlib.reload(get_node_info)
from get_node_info import get_fluid_prop

pyvista version used: 0.45.3


In [23]:

step = 100
from transformations import normalize_fluid_data


In [24]:
mesh = pv.read("VTK_Uin1_Re1000/cavity_0.vtk")
cell_cent = mesh.cell_centers()
cell_ID = mesh.cell_data["cellID"].astype(np.int64)

In [25]:
pts = cell_cent.points
x = pts[:, 0]
y = pts[:, 1]
z = pts[:, 2]
# on an unstructured grid cell centered we use interpolation to get the values of the variables at the grid instead of using the cell centered values.

cell_data = np.column_stack([cell_ID, x, y, z])
n_cells = np.shape(cell_data)[0]
xy = np.column_stack([x, y])

In [26]:
from edge_info import edges_from_faces
from edge_info import build_edge_feature

edge_info = edges_from_faces(mesh)
edge_attribute = build_edge_feature(mesh, edge_info)
# distance from edge to edge is used as feature

In [ ]:
from data_processor import GraphDataset

edge_index = torch.tensor(edge_info.T, dtype=torch.long)
edge_attr = torch.tensor(edge_attribute, dtype=torch.float32)
xy = torch.tensor(xy, dtype=torch.float32)
dataset = GraphDataset(cases, xy, edge_info, edge_attribute, step, normalize_fluid_data)

In [10]:
import torch
from torch_geometric.data import Data
"""
# node features: shape (N, F)
x_train = torch.tensor(node_data, dtype=torch.float32)
x_test = torch.tensor(node_data_next, dtype=torch.float32)

# edges: your edge_info is shape (E, 2)
edge_index = torch.tensor(edge_info.T, dtype=torch.long)   # (2, E)

# edge features: shape (E, 3)
edge_attr = torch.tensor(edge_attribute, dtype=torch.float32)
global_attr_train = torch.tensor([1,100], dtype=torch.float32)
global_attr_test = torch.tensor([1,1000], dtype=torch.float32)
data = Data(x=x_train, edge_index=edge_index, edge_attr=edge_attr,global_attr=global_attr_train)
data_plus = Data(x=x_test, edge_index=edge_index, edge_attr=edge_attr, global_attr=global_attr_test)
np.shape(edge_attr)"""

'\n# node features: shape (N, F)\nx_train = torch.tensor(node_data, dtype=torch.float32)\nx_test = torch.tensor(node_data_next, dtype=torch.float32)\n\n# edges: your edge_info is shape (E, 2)\nedge_index = torch.tensor(edge_info.T, dtype=torch.long)   # (2, E)\n\n# edge features: shape (E, 3)\nedge_attr = torch.tensor(edge_attribute, dtype=torch.float32)\nglobal_attr_train = torch.tensor([1,100], dtype=torch.float32)\nglobal_attr_test = torch.tensor([1,1000], dtype=torch.float32)\ndata = Data(x=x_train, edge_index=edge_index, edge_attr=edge_attr,global_attr=global_attr_train)\ndata_plus = Data(x=x_test, edge_index=edge_index, edge_attr=edge_attr, global_attr=global_attr_test)\nnp.shape(edge_attr)'

In [11]:
from torch_geometric.loader import DataLoader

loader = DataLoader(dataset, batch_size=2, shuffle=True)

In [12]:
import matplotlib.pyplot as plt
import numpy as np

def plot_degree(data):
    N = data.num_nodes
    src = data.edge_index[0]
    deg = torch.bincount(src, minlength=N).float()  # out-degree
    pos = data.x[:, :2].cpu().numpy()
    deg_np = deg.numpy()

    plt.figure()
    plt.scatter(pos[:,0], pos[:,1], c=deg_np, s=2)
    plt.gca().set_aspect('equal', 'box')
    plt.title("Node out-degree (should be ~3 interior, lower near boundaries)")
    plt.colorbar()
    plt.show()


In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(torch.__version__)

2.5.1+cu121


In [14]:
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import MessagePassing

class PoolModel(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        # Initial Graph Convolution
        self.conv1 = GCNConv(data.num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)

        # Regression head to predict your CFD values (e.g., Ux, Uy, P)
        self.out = torch.nn.Linear(hidden_channels, 5)

    def forward(self, data):
        # 1. Obtain node embeddings

        x, edge_index= data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)

        # 2. Node-level output (for the full 50,480 vector prediction)
        return self.out(x)

In [15]:
"""device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = PoolModel(64).to(device)
data = data.to(device)
data_plus = data_plus.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

model.train()
total_loss = 0
for epoch in range(100):

    optimizer.zero_grad()
    out = model(data)
    loss = F.mse_loss(out, data_plus.x)
    loss.backward()
    optimizer.step()
    total_loss += loss.item()

    # Print average loss for the epoch
    avg_loss = total_loss / (epoch+1)
    print(f"Epoch {epoch+1:03d} | Average Loss: {avg_loss:.6f}")"""

'device = torch.device(\'cuda\' if torch.cuda.is_available() else \'cpu\')\nmodel = PoolModel(64).to(device)\ndata = data.to(device)\ndata_plus = data_plus.to(device)\noptimizer = torch.optim.Adam(model.parameters(), lr=0.01)\n\nmodel.train()\ntotal_loss = 0\nfor epoch in range(100):\n\n    optimizer.zero_grad()\n    out = model(data)\n    loss = F.mse_loss(out, data_plus.x)\n    loss.backward()\n    optimizer.step()\n    total_loss += loss.item()\n\n    # Print average loss for the epoch\n    avg_loss = total_loss / (epoch+1)\n    print(f"Epoch {epoch+1:03d} | Average Loss: {avg_loss:.6f}")'

In [16]:
class GNNMessagePassing(MessagePassing):
    def __init__(self, node_input, edge_in, hidden_channels, global_att_dim):
        super(GNNMessagePassing, self).__init__(aggr='add')

        self.msg_mlp = Sequential(
            nn.Linear(2*hidden_channels + edge_in + global_att_dim, hidden_channels),
            nn.ReLU(),
            nn.Linear(hidden_channels, hidden_channels),
        )
        self.upd_mlp = Sequential(
            nn.Linear(2*hidden_channels, hidden_channels),
            nn.ReLU(),
            nn.Linear(hidden_channels, hidden_channels),
        )

    def forward(self, x, edge_index, edge_attr, global_attr):
        if global_attr.dim() == 2:
            global_attr = global_attr.squeeze(0)  # [G]

        E = edge_attr.size(0)  # 150998
        g_edge = global_attr.unsqueeze(0).expand(E, -1)
        return self.propagate(edge_index, x=x, edge_attr=edge_attr,global_attr=g_edge)

    def message(self,x_i, x_j,edge_attr,global_attr):
        combined = torch.cat([x_i, x_j, edge_attr, global_attr], dim=-1)
        return self.msg_mlp(combined)

    def update(self, aggr_out, x):
        combined = torch.cat([x, aggr_out], dim=-1)
        return self.upd_mlp(combined)

In [17]:
#using the message passing model

class FlowPredictor(torch.nn.Module):
    def __init__(self,hidden_channels,num_layers):
        super().__init__()
        self.num_layers = 10
        self.layers = torch.nn.ModuleList()

        for _ in range(self.num_layers):
            self.layers.append(GNNMessagePassing(node_input=hidden_channels, edge_in=3, hidden_channels=hidden_channels, global_att_dim=2)) ## what sort of layers do we have in our module list.
        self.encoder = nn.Linear(5, hidden_channels) # [x, y, u, v, p] -> 64
#        self.processor = GNNMessagePassing(node_input=hidden_channels, edge_in=3, hidden_channels=hidden_channels)
        self.decoder = nn.Linear(hidden_channels, 5) # hidden_channels -> [x, y, u, v, p]

    def forward(self, data):
        h = self.encoder(data.x)
        for layer in self.layers:
            h_update = layer(h, data.edge_index, data.edge_attr, data.global_attr)
            print(h_update)

        h = h + h_update
        return self.decoder(h)

    print(edge_attr.size())

torch.Size([150998, 3])


In [18]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = FlowPredictor(hidden_channels=64,num_layers=10).to(device)
#data = data.to(device)
#data_plus = data_plus.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

model.train()
total_loss = 0
total_samples = len(loader.dataset)
print(total_samples)
for epoch in range(100):
    for batch_idx in loader:
        optimizer.zero_grad()
        batch_idx = batch_idx.to(device)
        output, latent = model(batch_idx)
        loss = F.mse_loss(output, batch_idx)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(loader)
    if epoch % 10 == 0:
        print(f"Epoch {epoch+1:03d} | Average Loss: {avg_loss:.6f}")


10
xy: <class 'torch.Tensor'> torch.Size([50480, 2]) torch.float32
uvp: <class 'torch.Tensor'> torch.Size([50480, 3]) torch.float32
xy: <class 'torch.Tensor'> torch.Size([50480, 2]) torch.float32
uvp: <class 'torch.Tensor'> torch.Size([50480, 3]) torch.float32


RuntimeError: mat1 and mat2 shapes cannot be multiplied (301996x135 and 133x64)

In [ ]:
"lets put the model on eval mode now"

model.eval()
with torch.no_grad():
    data_plus.y = data.x[:, 2:5].clone()
    x_in = data_plus.x.clone()
    x_in[:, 2:5] = 0.0

    data_test_in = data_plus.clone()
    data_test_in.x = x_in
    data_test_in.global_attr = torch.tensor([1, 1000], device=device).float()

    pred = model(data_test_in)

test_loss = F.mse_loss(pred[:, 2:5], data_plus.y).item()
rel = torch.norm(pred[:,2:5] - data_plus.y) / torch.norm(data_plus.y)
print("Relative L2 error:", rel.item())
print("test mse:", test_loss)